# SCA through Dino LFP data

In [22]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append('/Users/ldliao/Research/Projects/spectralCA/')
sys.path.append('/Users/ldliao/Research/Projects/spectralCA/notebooks_visualization/')
# sys.path.append('/Users/Lauren/Voytek/spectralCV')
sys.path.append('/Users/rdgao/Documents/code/research/spectralCA')
sys.path.append('/Users/rdgao/Documents/code/research/neurodsp/')
# sys.path.append('/Users/rdgao/Documents/code/research/fooof/')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [23]:
import os
path = '/Users/ldliao/Research/Projects/spectralCA/results/dino/'

os.chdir(path) # DONT CHANGE PATH INSIDE THE NOTEBOOK, YOU"LL GET LOST. USE FULL PATH
import glob
from sca import sca
import neurodsp as ndsp

# the final product to import the visualization
from sca import sca_vis

In [24]:
%matplotlib inline

# imports
# from sca import access_nt as asc
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
import scipy.io as io
import pandas as pd

import neurodsp as ndsp
# from fooof import FOOOFGroup
# import sca

In [25]:
data_path = "/Users/ldliao/Research/Data/Dino/Dino_072114_mPFC_ephys.mat"
# data_path = "/Users/rdgao/Documents/data/CRCNS/fcx1/Dino_072114_mPFC/Dino_072114_mPFC_ephys.mat"
dino = io.loadmat(data_path, squeeze_me=True)

In [26]:
info_path = "/Users/ldliao/Research/Data/Dino/Dino_072114_mPFC_wakesleep.csv"
# info_path = "/Users/rdgao/Documents/data/CRCNS/fcx1/Dino_072114_mPFC/Dino_072114_mPFC_wakesleep.csv"
info = pd.read_csv(info_path,index_col='Unnamed: 0')

In [ ]:
info[-3:]

In [38]:
label = 'WakeSleep'
duration = list(zip(info[info['Label'] == label]['Start'].values, info[info['Label'] == label]['End'].values))
fs = 1250
analysis_param = {'nperseg': int(fs),
                 'noverlap': 0,
                 'spg_outlierpct': 4.,
                 'max_freq':200}

data = []
for dur in duration:
    start, end = dur
    
    # don't append like this because there will be a gap if there are multiple trials
    # of the same type. Compute sca separately
    data.append(dino['lfp'][:,start*fs:end*fs])
    
data = np.column_stack(data)

In [39]:
# initialize object with data and parameters
dino_sca = sca.SCA(analysis_param)
dino_sca.populate_ts_data(data, fs)

# compute all spectral attributes
dino_sca.compute_all_spectral()

# compute KS-test fit against null exponential hypothesis
dino_sca.compute_KS_expfit() # this takes a few minutes

chan_count, freq = dino_sca.psd.shape
chan_labels = list(map(str, range(chan_count)))

In [40]:
dino_sca.chan_labels = ['chan_'+chan for chan in chan_labels]
# save out
dino_sca.save_spec_vars('results/dino/'+label.lower(), save_spg=True)

In [42]:
dino_sca.chan_labels

['chan_0', 'chan_1', 'chan_2', 'chan_3']